In [285]:
import pandas as pd
import numpy as np
import random

In [286]:
def read_data(file):
    data = pd.read_excel(file, header=None)
    return data

In [287]:
data = read_data("JSSP-problems/basic-problem.xlsx")
data

,0,1,2
0,3,2,2
1,2,4,1
2,0,4,3


# CHROMOSOMA

Representat com una llista de tuples, on gada gen es una tupla. La primera posició de la tupla indica el job i la segona posició indica la màquina. 

In [288]:
params = {"population_size": 10, 
          "mutation_rate": 0.1, 
          "crossover_rate": 0.5, 
          "max_generations": 100}

In [289]:
def generate_inital_population(data, population_size):
    jobs = data.shape[0]
    machines = data.shape[1]
    population = []
    for _ in range(population_size):
        chromosome = []
        for job in range(jobs):
            for machine in range(machines):
                chromosome.append((job, machine))
        random.shuffle(chromosome)
        population.append(chromosome)
    return population


In [290]:
def fitness(chromosome, data):
    job_end_time = [0] * len(data)  # End time of each job
    machine_end_time = [0] * len(data[0])  # End time of each machine
    
    for task in chromosome:
        job, machine = map(int, task)
        start_time = max(job_end_time[job], machine_end_time[machine]) # Start time of the task
        job_end_time[job] = start_time + data[job][machine] # Update end time of job
        machine_end_time[machine] = start_time + data[job][machine] # Update end time of machine
    
    return max(machine_end_time)  # Makespan

In [291]:
def selection(population, fitness_values, tournament_size): # Tournament selection
    selected = set()
    
    while len(selected) < 2:  # Select 2 parents
        tournament = random.sample(list(enumerate(fitness_values)), tournament_size)
        winner = min(tournament, key=lambda x: x[1])[0] # Select the chromosome with the lowest fitness value
        selected.add(winner)

    selected = list(selected)
    parent1 = population[selected[0]]
    parent2 = population[selected[1]]
    return parent1, parent2

In [292]:
def crossover(parent1, parent2, probability): # Uniform crossover
    size = len(parent1)
    child1, child2 = [], []

    # Uniform crossover
    for i in range(size):
        if random.random() < probability:
            child1.append(parent1[i])
            child2.append(parent2[i])
        else:
            child1.append(parent2[i])
            child2.append(parent1[i])

    # Repair duplicated genes in children
    def repair(child, parent):
        used_genes = set(child)
        missing_genes = [gene for gene in parent if gene not in used_genes]
        for i in range(size):
            if child.count(child[i]) > 1:  # Check if the gene is duplicated
                child[i] = missing_genes.pop(0)  # Substitute duplicated gene with a missing gene
        return child

    child1 = repair(child1, parent1)
    child2 = repair(child2, parent2)

    return child1, child2

In [293]:
def mutation(chromosome, mutation_probability):
    for i in range(len(chromosome)):
        if random.random() < mutation_probability:
            job, machine = chromosome[i]
            if random.random() < 0.5:
                job = random.randint(0, max(job for job, _ in chromosome))
            else:
                machine = random.randint(0, max(machine for _, machine in chromosome))
            chromosome[i] = (job, machine)
    return chromosome

In [294]:
def gen_algorithm(data, params):
    population_size = params["population_size"]
    mutation_rate = params["mutation_rate"]
    crossover_rate = params["crossover_rate"]
    max_generations = params["max_generations"]
    tournament_size = 4

    population = generate_inital_population(data, population_size)
    fitness_values = [fitness(chromosome, data) for chromosome in population]
    print(fitness_values)
    for generation in range(max_generations):
        new_population = []
        for _ in range(population_size):
            # Selection
            parent1, parent2 = selection(population, fitness_values, tournament_size)
            print(parent1)
            print(parent2)
            # Crossover
            child1, child2 = crossover(parent1, parent2, crossover_rate)
            # Mutation
            print(child1)
            print(child2)
            break
        break
    

In [295]:
gen_algorithm(data, params)

[18, 13, 14, 16, 14, 16, 16, 14, 12, 15]
[(0, 2), (2, 0), (0, 1), (2, 2), (1, 1), (2, 1), (0, 0), (1, 2), (1, 0)]
[(2, 1), (0, 0), (2, 2), (1, 2), (1, 1), (0, 2), (2, 0), (0, 1), (1, 0)]
[(0, 2), (0, 0), (2, 2), (0, 1), (1, 1), (2, 1), (2, 0), (1, 2), (1, 0)]
[(2, 1), (2, 0), (1, 2), (2, 2), (1, 1), (0, 2), (0, 0), (0, 1), (1, 0)]
